In [1]:

import numpy as np
import torch

import sys
import os
import scipy.io as sio

#print(os.path.dirname(os.path.abspath("")))
#sys.path.append(os.path.dirname(os.path.abspath("")))

cwd = os.getcwd()

#sys.path.append(cwd + '/my_directory')

sys.path.append(cwd)

import matplotlib.pyplot as plt

# General imports
import numpy as np
import torch

# DeePyMoD imports
from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.data.burgers import burgers_delta
from deepymod.model.constraint import LeastSquares, Ridge, STRidgeCons
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.sparse_estimators import Threshold, STRidge
from deepymod.training import train
#from deepymod.training.training_2 import train

from deepymod.training.sparsity_scheduler import Periodic, TrainTest, TrainTestPeriodic

#from deepymod.data.data_set_preparation import DatasetPDE, pde_data_loader

import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scipy.io import loadmat

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(50)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



cuda


In [2]:
def create_data():
    phi_data = np.load("data/data_ujjal.npy")
    phi_data = phi_data[14000:,:,:]
    #phi_data = phi_data[np.random.choice(phi_data.shape[0], size=100),:,:] 
    #for i in range(len(phi_data)):
    #    phi_data_sampled = phi_data[:,np.random.choice(phi_data.shape[1], size=250),:]
    #    phi_data_sampled = phi_data_sampled[:, :, np.random.choice(phi_data.shape[2], size=200)]
    
    phi_data_sampled = phi_data 
    phi_data_sampled = np.transpose(phi_data_sampled, (1, 0, 2))
    phi_data_sampled = np.transpose(phi_data_sampled, (0, 2, 1))
    t_num = phi_data_sampled.shape[2]
    x_num = phi_data_sampled.shape[0]
    y_num = phi_data_sampled.shape[1]
    ## preparing and normalizing the input and output data
    t = np.linspace(0, 1, t_num)
    x = np.linspace(-1, 1, x_num)
    y = np.linspace(-1, 1, y_num)
    X, Y, T = np.meshgrid(x, y, t, indexing="ij")
    X_star = np.hstack((X.flatten()[:, None], Y.flatten()[:, None], T.flatten()[:, None]))
    u_star = phi_data.flatten()[:, None]
    print("The coodinates have shape {}".format(X_star.shape))
    print("The data has shape {}".format(u_star.shape))
    
    X_star = torch.tensor(X_star.astype(float), requires_grad = True)
    u_star = torch.tensor(u_star.astype(float), requires_grad = True)
    #data = loadmat("data/advection_diffusion.mat")
    #usol = np.real(data["Expression1"]).astype("float32")
    #usol = usol.reshape((51, 51, 61, 4))
    #x_v = usol[:,:,:,0]
    #y_v = usol[:,:,:,1]
    #t_v = usol[:,:,:,2]
    #u_v = usol[:,:,:,3]
    #coords = torch.from_numpy(np.stack((t, x, y), axis=-1))
    #data = torch.from_numpy(usol[:, :, :, 3]).unsqueeze(-1)
    # alternative way of providing the coordinates
    # coords = torch.from_numpy(np.transpose((t_v.flatten(), x_v.flatten(), y_v.flatten())))
    # data = torch.from_numpy(usol[:, :, :, 3].reshape(-1,1))
    #print("The coodinates have shape {}".format(coords.shape))
    #print("The data has shape {}".format(data.shape))
    return X_star, u_star

X_star, u_star = create_data()


The coodinates have shape (20400000, 3)
The data has shape (20400000, 1)


In [ ]:
phi_data = np.load("data/data_ujjal.npy")
phi_data = phi_data[14400:,:,:]

phi_data.shape


import numpy as np

# Assuming your reshaped dataset is stored in 'data_matrix'
data_matrix = np.reshape(phi_data, (phi_data.shape[0], 250 * 200))

# Apply SVD
U, s, VT = np.linalg.svd(data_matrix)



In [ ]:

dataset = Dataset(
    create_data,
    preprocess_kwargs={
        "noise_level": 0.05,
        "normalize_coords": True,
        "normalize_data": True,
    },
    subsampler=Subsample_random,
    subsampler_kwargs={"number_of_samples": 5000},
    device=device,
)

coords = dataset.get_coords().detach().cpu()
data = dataset.get_data().detach().cpu()


In [ ]:
print(data)